In [1]:
import hdlparse.verilog_parser as vlog
import re
import random

filename = "Module1.v"
ports = []
if_cases_conditions = set()
always_conditions = set()
case_conditions = set()

In [2]:
def vector_size(vector) :
    left = vector[1]
    right = vector[-2]
    return (abs(int(left) - int(right))+1)

In [3]:
def vector_sizeR(line):
	if re.search(r'\[', line):
		lval = re.findall(r'\[(.*):', line)[0]
		rval = re.findall(r'\[.*:(.*)\]', line)[0]
		return (abs(int(rval) - int(lval)) + 1)
	return 1

## Open and read verilog file

In [4]:
vlog_ex = vlog.VerilogExtractor()
with open(filename, 'rt') as fh:
    code = fh.read()
vlog_mods = vlog_ex.extract_objects_from_source(code)
vlog_mods = vlog_ex.extract_objects(filename)

## Add inputs and outputs to list as dictionary

In [5]:


for m in vlog_mods:
    print('Module "{}":'.format(m.name))
    for p in m.ports:
        if (p.data_type).find('[') != -1 :
            ports.append({"name":p.name ,"mode":p.mode ,"data_type":(p.data_type).split()[0],"length":vector_size((p.data_type).split()[1]) })
        else :
            ports.append({"name":p.name ,"mode":p.mode ,"data_type":p.data_type, "length": 1})
        print('\t{:20}{:8}{}'.format(p.name, p.mode, p.data_type))


Module "Test1":
	HasAccess           input   wire
	Input_Pin           input   reg [3:0]
	message             output  reg [3:0]
	Welcome             output  reg


In [6]:
print(ports)

[{'name': 'HasAccess', 'mode': 'input', 'data_type': 'wire', 'length': 1}, {'name': 'Input_Pin', 'mode': 'input', 'data_type': 'reg', 'length': 4}, {'name': 'message', 'mode': 'output', 'data_type': 'reg', 'length': 4}, {'name': 'Welcome', 'mode': 'output', 'data_type': 'reg', 'length': 1}]


# Control flow parsing

### Parsing if conditions

In [7]:
def parse_if(line) :
    if line.find("||") != -1 or line.find("&&") != -1:
        for port in ports :
            if port["mode"]=="input" :
                if line.find(port["name"]) != -1 :
                    if_cases_conditions.add(port["name"])
    else :
        str = re.split("if\s*\(\s*!?",line)[1]
        str = re.split("\s*\)",str)[0]
        if_cases_conditions.add(str)
        

### Parse always statement

In [8]:
def parse_always(line) :
    if line.find("||") != -1 or line.find("&&") != -1 or line.find("or") != -1 or line.find("and") != -1:
        for port in ports :
            if port["mode"]=="input" :
                if line.find(port["name"]) != -1 :
                    always_conditions.add(port["name"])
    else :
        str = re.split("always\s*@\s*\(\s*!?",line)[1]
        str = re.split("\s*\)",str)[0]
        always_conditions.add(str)

### Parse case statement

In [9]:
def parse_case(line) :
    str = re.split("case\s*\(\s*",line)[1]
    str = re.split("\s*\)",str)[0]
    case_conditions.add(str)

In [10]:
file = open(filename,"rt")
for line in file :
    if re.search("if\s*\(\s*\w+",line) :
        parse_if(line)
    if re.search("always\s*@\s*\(\s*\w+\s*",line):
        parse_always(line)
    if re.search("case\s*\(\s*\w+\s*\)",line) :
        parse_case(line)
print(if_cases_conditions)
print(always_conditions)
print(case_conditions)

{"message == 4'b1111 & Input_Pin == pin", 'HasAccess'}
{'HasAccess'}
set()


# Generator

### Output on console

In [11]:

# time_delay = 10
# print('module ' + m.name + '_tb();')
# for port in ports:
#     if port["mode"].find('input', 0, 5) != -1:
#         print('    ', port["mode"].replace('input', 'reg'), ';')
#     elif port["mode"].find('output', 0, 6) != -1:
#         print('    ', port["mode"].replace('output', 'wire'),';')
#     # else:
#     #     print('    ', port, ';')
# print('//initial block\ninitial','\n  begin', '\n    //initial values')
# for i in range (7):
#     if(i != 0):
#         print('  #'+ str(time_delay))
#     for line in ports:
#         if line["mode"] == 'input':
#             val = line["length"]
#             print('    ',line["name"], '=', str(val),'\'b'+ bin(random.randint(0, pow(2,val)-1))[2:].zfill(val))

# print('  $finish;','\nend\n')
# print(m.name, 'DUT (')
# for i,line in enumerate(ports, 0):
#     print('  ',line["name"], end='')
#     if i != len(ports)-1:
#         print(',')
#     else:
#         print('\n);')

# print('\nendmodule')


## File generation

### generate vector function if port is vector

In [12]:
def generate_vector(port_length):
    if port_length == 1:
        return ' '
    else:
        return (' [' + str(port_length - 1) + ':0] ')

### Add testbench content

In [13]:
def testbench_header_and_ports():
    content = ""
    content += ('module ' + m.name + '_tb();\n')
    for port in ports:
    # if port["mode"].find('wire'):
    #     port.replace('wire', ' ')
        if port["mode"].find('input', 0, 5) != -1:
            content += ('    ' + port["mode"].replace('input', 'reg') +  generate_vector(port["length"]) + port["name"] + ';\n')
        elif port["mode"].find('output', 0, 6) != -1:
            content += ('    ' + port["mode"].replace('output', 'wire') + generate_vector(port["length"]) + port["name"] +  ';\n')
    return content

In [14]:
def testbench_initial_block(time_delay):
    content = ""
    content += ('//initial block\ninitial\n  begin\n    //initial values\n')
    maxPortSize = 3
    for port in ports :
        maxPortSize = max(port["length"],maxPortSize)
    iterationCounter = int(pow(2,maxPortSize)/2)
    for i in range (iterationCounter):
        if(i != 0):
            content += ('  #'+ str(time_delay)+'\n')
        for line in ports:
            if line["mode"] == 'input':
                val = line["length"]
                content += ('    ' + line["name"] + ' = '+ str(val) + '\'b'+ bin(random.randint(0, pow(2,val)-1))[2:].zfill(val)+'\n')

    content += ('  $finish;\nend\n')
    return content

In [15]:
def testbench_design_instance():
    content = ""
    content += ('// instaniate design instance\n  ' + m.name + ' DUT (\n')
    for i,line in enumerate(ports, 0):
        content += ('    .' + (line["name"]).replace('_tb', '') + '(' + line["name"]+')')
        if i != len(ports)-1:
            content += (',\n')
        else:
            content += ('\n);\n')

    content += ('\nendmodule')

    return content

In [16]:
# content = ""
# time_delay = 10 #input or not
# content += ('module ' + m.name + '_tb();\n')
# for port in ports:
#     # if port["data_type"].find('wire', 0, 12):
#     #     port["data_type"].replace('wire', '')
#         if port["mode"].find('input', 0, 5) != -1:
#             content += ('    ' + port["mode"].replace('input', 'reg') + ' ' + port["name"] +';\n')
#         elif port["mode"].find('output', 0, 6) != -1:
            
#             content += ('    ' + port["mode"].replace('output', 'wire') +' ' + port["name"] +';\n')
#         # else:
#         #     content += ('    ', port, ';\n')
# content += ('//initial block\ninitial\n  begin\n    //initial values\n')
# maxPortSize = 3
# for port in ports :
#     maxPortSize = max(port["length"],maxPortSize)
# iterationCounter = int(pow(2,maxPortSize)/2)
# for i in range (iterationCounter):
#     if(i != 0):
#         content += ('  #'+ str(time_delay)+'\n')
#     for line in ports:
#         if line["mode"] == 'input':
#             val = line["length"]
#             content += ('    ' + line["name"] + ' = '+ str(val) + '\'b'+ bin(random.randint(0, pow(2,val)-1))[2:].zfill(val)+'\n')

# content += ('  $finish;\nend\n')
# content += (m.name + ' DUT (\n')
# for i,line in enumerate(ports, 0):
#     content += ('  .' + (line["name"]).replace('_tb', '') + '(' + line["name"]+')')
#     if i != len(ports)-1:
#         content += (',\n')
#     else:
#         content += ('\n);\n')

# content += ('\nendmodule')

## Testbench file generation

In [17]:
def generate_output_file(file_content):
    with open('output.v', 'w') as f:
        f.write(file_content)

In [18]:
time_delay = 10 #input or not
test_bench_content = ""
test_bench_content += testbench_header_and_ports() + testbench_initial_block(time_delay) + testbench_design_instance()
generate_output_file(test_bench_content)